In [27]:
from __future__ import print_function
# Preprocessing
from sklearn import preprocessing
## load_to_folder
from os import listdir
from os.path import isdir, isfile, join, getsize
import numpy as np
import matplotlib.pyplot as plt
## libROSA
import librosa
import librosa.display
import IPython.display
## Data management
import pickle
#Machine Learning
from sklearn import svm
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

import warnings
warnings.filterwarnings("ignore")

In [2]:
pathAudio = "/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data"
files = librosa.util.find_files(pathAudio, ext=['mp3']) 
files = np.asarray(files)

for file_name in files: 
    y, sr = librosa.load(file_name, sr=None)
#IPython.display.Audio(data=y, rate=sr)

In [3]:
files


array(['/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/De Lorra - Slow Drip.mp3',
       '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/Flume - Wall Fuck (Jey Vazz Remix).mp3',
       '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/HOME - Resonance.mp3',
       '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/kd - Electronic Memories (feat. Mickey Kojak).mp3',
       '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/Leon Lour - All Our Dreams [Music Video - 14].mp3',
       '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/MACINTOSH PLUS - リサフランク420  現代のコンピュー (VANTAGE Remix).mp3',
       '/home/arthorius/Documents/UNB/FSI/Projetos

In [4]:
folderpath = "/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data"
folders = [f for f in listdir(folderpath) if isdir(folderpath+"/"+f) and f[0]!='.']
files_all = []
for folder in folders:
    files = [f for f in listdir(folderpath+"/"+folder) if isfile(join(folderpath+"/"+folder+"/"+f)) and f[0]!='.']
    for file_name in files:
        files_all.append(folderpath+"/"+folder+"/"+file_name)

In [5]:
files_all

['/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/MACINTOSH PLUS - リサフランク420  現代のコンピュー (VANTAGE Remix).mp3',
 '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/Voyage - Dynamic.mp3',
 '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/Unlike Pluto - Run, Bobby, Run.mp3',
 '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/Leon Lour - All Our Dreams [Music Video - 14].mp3',
 '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/b/Unlike Pluto - Late Bloomer.mp3',
 '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/Flume - Wall Fuck (Jey Vazz Remix).mp3',
 '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classi

# FEATURES

In [14]:
#folderpath = '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/De Lorra - Slow Drip.mp3'
folderpath = '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/kd - Electronic Memories (feat. Mickey Kojak).mp3'
#folderpath = '/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/a/HOME - Resonance.mp3'
y, sr = librosa.load(folderpath, duration = 120)
S, phase = librosa.magphase(librosa.stft(y))



In [15]:
# Low energy is the percentage of audio with less RMS energy than the average of the entire
# waveform. Each sample is divided into frames for the purpose of the computation. This feature is
# used as an effective measure of amplitude distribution
# https://librosa.github.io/librosa/generated/librosa.feature.rmse.html
rms_S = librosa.feature.rmse(S=S)
rms_S = np.ravel(rms_S)
rms_S_mean = np.mean(rms_S, dtype=np.float64)
low_energy_frame_count = 0
for rms_frame in rms_S:
    if rms_frame < rms_S_mean:
        low_energy_frame_count+=1;
low_energy_percentage = (100*low_energy_frame_count)/rms_S.size
low_energy_percentage = np.array([low_energy_percentage])
print("low_energy_percentage = " + format(low_energy_percentage) + "%")
low_energy_percentage.shape
#plt.figure()
#plt.subplot(2, 1, 1)
#plt.semilogy(rms_S.T, label='RMS Energy')
#plt.xticks([])
#plt.xlim([0, rms_S.shape[-1]])
#plt.legend(loc='best')
#plt.subplot(2, 1, 2)
#librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time')
#plt.title('log Power spectrogram')
#plt.tight_layout()

low_energy_percentage = [56.25]%


(1,)

In [16]:
# The zero-crossing rate is calculated as the number of times a signal crosses the time axis. Zero-
# crossings can be used to measure the noisiness of audio signals.
# https://librosa.github.io/librosa/generated/librosa.feature.zero_crossing_rate.html#librosa.feature.zero_crossing_rate
# https://librosa.github.io/librosa/generated/librosa.core.zero_crossings.html#librosa.core.zero_crossings
zero_crossings = np.ravel(librosa.core.zero_crossings(y))
zcs_indexes = np.ravel(np.nonzero(zero_crossings))
zero_crossing_rate = (100*zcs_indexes.size)/zero_crossings.size
zero_crossing_rate = np.array([zero_crossing_rate])
print("zero_crossing_rate = " + format(zero_crossing_rate) + "%")
zero_crossing_rate.shape

zero_crossing_rate = [19.21027967]%


(1,)

In [17]:
# Tempo is the speed at which a piece of music is played, measured in beats per minute (bpm). Essentially,
# autocorrelation is computed using an optimized FFT approach, resulting in peaks at intervals of
# the tempo. This effect is sharpened for music samples by taking phase into account and using
# entropy as a measure of “peakiness.” The strongest peaks are then selected from a window of the
# range of appropriate tempos.
# https://librosa.github.io/librosa/generated/librosa.beat.tempo.html#librosa.beat.tempo
tempo = np.ravel(librosa.beat.tempo(y=y, sr=sr))
print("tempo = " + format(tempo) + " bpm")
tempo.shape

tempo = [99.38401442] bpm


(1,)

In [19]:
# Spectral rolloff is the frequency below which 85% of magnitude distribution lies; it is used as a
# measure of spectral shape.
# https://librosa.github.io/librosa/generated/librosa.feature.spectral_rolloff.html#librosa.feature.spectral_rolloff
spectral_rolloff = librosa.feature.spectral_rolloff(S=S, roll_percent=0.85)
spectral_rolloff = np.ravel(spectral_rolloff)
print("spectral_rolloff = " + format(spectral_rolloff))

if(spectral_rolloff.shape != (5168, )):
    spectral_rolloff.resize((5168, ), refcheck=False)
    
spectral_rolloff.shape
#plt.figure()
#plt.subplot(2, 1, 2)
#plt.semilogy(spectral_rolloff.T, label='Roll-off frequency')
#plt.ylabel('Hz')
#plt.xticks([])
#plt.xlim([0, spectral_rolloff.shape[-1]])
#plt.legend()
#plt.subplot(2, 1, 2)
#librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time')
#plt.title('log Power spectrogram')
#plt.tight_layout()



spectral_rolloff = [5189.50195312 2734.71679688 3273.046875   ... 7902.68554688 7870.38574219
 7945.75195312]


(5168,)

In [20]:
# Brightness (or high-frequency energy) is the amount of energy above a selected threshold,
# typically somewhere between 500 and 3000 Hz for speech and audio. Brightness is a primary
# measure of timbre (sharpness vs. softness).

### TODO

print("brightness = " + format(brightness))


NameError: name 'brightness' is not defined

In [21]:
# Mel-frequency cepstral coefficients (MFCCs) are designed to succinctly represent a smoothed
# spectrum of an audio signal. 
# MFCCs compactly summarize the mel-spectrum response (the smoothed, modified spectral envelope).
# Instead of the 13 coefficients that are typically used for speech recognition applications, the first
# 5 MFCCs were chosen for this feature for maximum effectiveness in music per Tzanetakis and Cook
# https://librosa.github.io/librosa/generated/librosa.feature.mfcc.html#librosa.feature.mfcc
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=5)
print("mfcc = " + format(mfccs))

if(mfccs.shape != (5, 5168)):
    mfccs.resize((5, 5168), refcheck=False)
mfcc0 = mfccs[0]
mfcc1 = mfccs[1]
mfcc2 = mfccs[2]
mfcc3 = mfccs[3]
mfcc4 = mfccs[4]

#plt.figure(figsize=(10, 4))
#librosa.display.specshow(mfccs, x_axis='time')
#plt.colorbar()
#plt.title('MFCC')
#plt.tight_layout()
mfccs.shape

mfcc = [[-461.0887237  -308.42172439 -194.45383742 ...   87.98302647
    86.87062555   74.59293244]
 [   0.          102.31194137  120.65952649 ...   39.24597596
    40.88074364   41.8082212 ]
 [   0.          -23.37352738  -70.87955839 ...   12.34556704
     6.14307695    5.42898807]
 [   0.          -17.87283169    6.32876508 ...    9.51426799
    10.76049748    3.95153632]
 [   0.          -15.53974365  -31.15213806 ...   -5.29558007
    -2.46784305   -1.44812835]]


(5, 5168)

### Additional Features

In [22]:
# Spectral Bandwidth
# https://librosa.github.io/librosa/generated/librosa.feature.spectral_bandwidth.html#librosa.feature.spectral_bandwidth
spec_bw = librosa.feature.spectral_bandwidth(S=S)
if(spec_bw.shape != (5168, )):
    spec_bw.resize((5168, ), refcheck=False)
print("spec_bw = " + format(spec_bw))

# plt.figure()
# plt.subplot(2, 1, 1)
# plt.semilogy(spec_bw.T, label='Spectral bandwidth')
# plt.ylabel('Hz')
# plt.xticks([])
# plt.xlim([0, spec_bw.shape[-1]])
# plt.legend()
# plt.subplot(2, 1, 2)
# librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time')
# plt.title('log Power spectrogram')
# plt.tight_layout()
spec_bw.shape


spec_bw = [2657.22151318 1632.58120403 1448.8794553  ... 3107.32265993 3068.11544394
 3119.06021778]


(5168,)

In [23]:
# Spectral Contrast
# https://librosa.github.io/librosa/generated/librosa.feature.spectral_contrast.html#librosa.feature.spectral_contrast
spec_contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
print("spec_contrast = " + format(spec_contrast))

if(spec_contrast.shape != (7, 5168)):
    spec_contrast.resize((7, 5168), refcheck=False)
spec_contrast0 = spec_contrast[0]
spec_contrast1 = spec_contrast[1]
spec_contrast2 = spec_contrast[2]
spec_contrast3 = spec_contrast[3]
spec_contrast4 = spec_contrast[4]
spec_contrast5 = spec_contrast[5]
spec_contrast6 = spec_contrast[6]

# plt.figure()
# plt.subplot(2, 1, 1)
# librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log')
# plt.colorbar(format='%+2.0f dB')
# plt.title('Power spectrogram')
# plt.subplot(2, 1, 2)
#librosa.display.specshow(contrast, x_axis='time')
#plt.colorbar()
#plt.ylabel('Frequency bands')
#plt.title('Spectral contrast')
#plt.tight_layout()
spec_contrast.shape


spec_contrast = [[17.74933453 17.97216958 13.73742659 ... 21.76272447 18.10416458
  18.41939689]
 [12.43791624 12.39651449 15.31356104 ... 12.73142058  9.34739149
  11.61130694]
 [24.34594504 10.98845353 13.17329595 ... 21.64806282 16.10457251
  17.59079631]
 ...
 [22.38485272 17.15886224 19.0230702  ... 12.75029152 16.38200865
  17.66003877]
 [18.70692447 19.1730511  21.90047662 ... 17.01195982 18.24503654
  15.74527402]
 [27.80939995 27.56415585 29.33331022 ... 34.64153593 34.35446125
  35.29125395]]


(7, 5168)

In [24]:
# Spectral Centroid
# https://librosa.github.io/librosa/generated/librosa.feature.spectral_centroid.html#librosa.feature.spectral_centroid
spec_centroid = librosa.feature.spectral_centroid(S=S)
print("spec_centroid = " + format(spec_centroid))
spec_centroid
if(spec_centroid.shape != (5168, )):
    spec_centroid.resize((5168, ), refcheck=False)
    
#plt.figure()
#plt.subplot(2, 1, 1)
#plt.semilogy(spec_centroid.T, label='Spectral centroid')
#plt.ylabel('Hz')
#plt.xticks([])
#plt.xlim([0, spec_centroid.shape[-1]])
#plt.legend()
#plt.subplot(2, 1, 2)
#librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time')
#plt.title('log Power spectrogram')
#plt.tight_layout()

spec_centroid.resize((5168,), refcheck=False)
spec_centroid.shape

spec_centroid = [[2165.46977401 1525.29207632 1586.42342034 ... 4052.96119511
  3968.06581501 4043.78493549]]


(5168,)

## TESTS

In [25]:
data = np.concatenate([low_energy_percentage, zero_crossing_rate, tempo, spectral_rolloff, mfcc0, mfcc1, mfcc2, mfcc3, mfcc4, spec_bw, spec_contrast0, spec_contrast1, spec_contrast2, spec_contrast3, spec_contrast4, spec_contrast5, spec_contrast6])

In [29]:
data.shape
# data.reshape(-1, 1)
data


array([56.25      , 19.21027967, 99.38401442, ..., 34.64153593,
       34.35446125, 35.29125395])

In [207]:

## TESTE SVM COM X ARBITRÁRIO, TENDO NELE FEATURE COM LISTA VARIÁVEL 
## OS TAMANHOS DAS FEATURES ESTAO DIFERENTES!! DEVE TESTAR ISSO! PROCURE UM PADRÃO MATEMATICO PARA IGUALAR TUDO (OU NAO)

from sklearn import svm
clf = svm.SVC()

In [208]:
clf.fit(data, [1])

ValueError: Expected 2D array, got 1D array instead:
array=[56.25       19.21027967 99.38401442 ... 34.64153593 34.35446125
 35.29125395].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

# PCA

In [2]:
def get_features_from_mp3(folderpath, folder, file_name):
    pathAudio = folderpath+"/"+folder+"/"+file_name
    ## Get only 2 minutes of the song. Can be enough for the classification.
    y, sr = librosa.load(pathAudio, sr=None, offset = 30, duration = 120)
    S, phase = librosa.magphase(librosa.stft(y))
    
    #Features:
    # Low energy is the percentage of audio with less RMS energy than the average of the entire
    # waveform. Each sample is divided into frames for the purpose of the computation. This feature is
    # used as an effective measure of amplitude distribution
    # https://librosa.github.io/librosa/generated/librosa.feature.rmse.html
    rms_S = librosa.feature.rmse(S=S)
    rms_S = np.ravel(rms_S)
    rms_S_mean = np.mean(rms_S, dtype=np.float64)
    low_energy_frame_count = 0
    for rms_frame in rms_S:
        if rms_frame < rms_S_mean:
            low_energy_frame_count+=1;
    low_energy_percentage = (100*low_energy_frame_count)/rms_S.size
    low_energy_percentage = np.array([low_energy_percentage])
    
    # The zero-crossing rate is calculated as the number of times a signal crosses the time axis. Zero-
    # crossings can be used to measure the noisiness of audio signals.
    # https://librosa.github.io/librosa/generated/librosa.feature.zero_crossing_rate.html#librosa.feature.zero_crossing_rate
    # https://librosa.github.io/librosa/generated/librosa.core.zero_crossings.html#librosa.core.zero_crossings
    zero_crossings = np.ravel(librosa.core.zero_crossings(y))
    zcs_indexes = np.ravel(np.nonzero(zero_crossings))
    zero_crossing_rate = (100*zcs_indexes.size)/zero_crossings.size
    zero_crossing_rate = np.array([zero_crossing_rate])
    
    # Tempo is the speed at which a piece of music is played, measured in beats per minute (bpm). Essentially,
    # autocorrelation is computed using an optimized FFT approach, resulting in peaks at intervals of
    # the tempo. This effect is sharpened for music samples by taking phase into account and using
    # entropy as a measure of “peakiness.” The strongest peaks are then selected from a window of the
    # range of appropriate tempos.
    # https://librosa.github.io/librosa/generated/librosa.beat.tempo.html#librosa.beat.tempo
    tempo = np.ravel(librosa.beat.tempo(y=y, sr=sr))
    
    # Spectral rolloff is the frequency below which 85% of magnitude distribution lies; it is used as a
    # measure of spectral shape.
    # https://librosa.github.io/librosa/generated/librosa.feature.spectral_rolloff.html#librosa.feature.spectral_rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(S=S, roll_percent=0.85)
    if(spectral_rolloff.shape != (5168, )):
        spectral_rolloff.resize((5168, ), refcheck=False)
    
    # Mel-frequency cepstral coefficients (MFCCs) are designed to succinctly represent a smoothed
    # spectrum of an audio signal. 
    # MFCCs compactly summarize the mel-spectrum response (the smoothed, modified spectral envelope).
    # Instead of the 13 coefficients that are typically used for speech recognition applications, the first
    # 5 MFCCs were chosen for this feature for maximum effectiveness in music per Tzanetakis and Cook
    # https://librosa.github.io/librosa/generated/librosa.feature.mfcc.html#librosa.feature.mfcc
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=5)
    mfcc0 = np.ravel(mfccs[0])
    mfcc1 = np.ravel(mfccs[1])
    mfcc2 = np.ravel(mfccs[2])
    mfcc3 = np.ravel(mfccs[3])
    mfcc4 = np.ravel(mfccs[4])
    
    if(mfcc0.shape != (5168, )):
        mfcc0.resize((5168, ), refcheck=False)
    
    if(mfcc1.shape != (5168, )):
        mfcc1.resize((5168, ), refcheck=False)
    
    if(mfcc2.shape != (5168, )):
        mfcc2.resize((5168, ), refcheck=False)
    
    if(mfcc3.shape != (5168, )):
        mfcc3.resize((5168, ), refcheck=False)
    
    if(mfcc4.shape != (5168, )):
        mfcc4.resize((5168, ), refcheck=False)
        
    # Spectral Bandwidth
    # https://librosa.github.io/librosa/generated/librosa.feature.spectral_bandwidth.html#librosa.feature.spectral_bandwidth
    spec_bw = librosa.feature.spectral_bandwidth(S=S)
    if(spec_bw.shape != (5168, )):
        spec_bw.resize((5168, ), refcheck=False)
    
    # Spectral Contrast
    # https://librosa.github.io/librosa/generated/librosa.feature.spectral_contrast.html#librosa.feature.spectral_contrast
    spec_contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    if(spec_contrast.shape != (7, 5168)):
        spec_contrast.resize((7, 5168), refcheck=False)
    spec_contrast0 = spec_contrast[0]
    spec_contrast1 = spec_contrast[1]
    spec_contrast2 = spec_contrast[2]
    spec_contrast3 = spec_contrast[3]
    spec_contrast4 = spec_contrast[4]
    spec_contrast5 = spec_contrast[5]
    spec_contrast6 = spec_contrast[6]
    # Spectral Centroid
    # https://librosa.github.io/librosa/generated/librosa.feature.spectral_centroid.html#librosa.feature.spectral_centroid
    spec_centroid = librosa.feature.spectral_centroid(S=S)
    if(spec_centroid.shape != (5168, )):
        spec_centroid.resize((5168, ), refcheck=False)
    
    data = np.concatenate([low_energy_percentage, zero_crossing_rate, tempo, spectral_rolloff, mfcc0, mfcc1, mfcc2, mfcc3, mfcc4, spec_bw, spec_contrast0, spec_contrast1, spec_contrast2, spec_contrast3, spec_contrast4, spec_contrast5, spec_contrast6])
    return data
    

In [3]:
def load_from_folder(folderpath):
    corpus = {}
    folders = [f for f in listdir(folderpath) if isdir(folderpath+"/"+f) and f[0]!='.']
    data_all=[]
    target_all=[]
    names_all=[]
    folder_idx = 0
    for folder in folders:
        files = [f for f in listdir(folderpath+"/"+folder) if isfile(join(folderpath+"/"+folder, f)) and f[0]!='.']
        for file_name in files:
            data = get_features_from_mp3(folderpath, folder, file_name)
            data_all.append(data)
            target_all.append(folder_idx)
        corpus['data'] = data_all
        corpus['target'] = target_all
        
        names_all.append(folder)
        
        folder_idx = folder_idx + 1
    corpus['names'] = names_all
    return corpus

In [7]:
pkl_path = "/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data/corpus.pkl"
corpus = {}
# https://sebastianraschka.com/Articles/2014_pca_step_by_step.html#generating-some-3-dimensional-sample-data
# 1) Take the whole dataset ignoring class labels
if (isfile(pkl_path)):
    if (getsize(pkl_path)) > 0:
        pkl_file = open(pkl_path, 'rb')
        corpus = pickle.load(pkl_file)
        pkl_file.close()
    else:
        print("File is empty.")
else:
    # audio_folder_path: absolute directory path. If you don't know how to get it, 
    # open folder in terminal and do the following commands:
    # python3
    # import os
    # os.getcwd()
    # then copy directory path
    audio_folder_path = "/home/arthorius/Documents/UNB/FSI/Projetos/FSI - Projeto Final/ml_electronic_subgenre_classification/data"
    corpus = load_from_folder(audio_folder_path)
    pickle_save = open(pkl_path, 'wb')
    pickle.dump(corpus, pickle_save)
    pickle_save.close

corpus_data = preprocessing.normalize(corpus['data'])

In [32]:
corpus_data.shape

(323, 72355)

# ---------------------------------------------------------------

In [30]:
# 2) Computing the d-dimensional mean vector
mean_vector = [];
for i in range(corpus_data.shape[0]):
    mean_i = np.mean(corpus_data[i])
    mean_vector.append(mean_i)
mean_vector

[0.0011788056880697062,
 0.0013327977636035614,
 0.0013430211528967831,
 0.001200773566808282,
 0.0012322862642660014,
 0.001136686929060812,
 0.0013109750870819122,
 0.0010894045070223518,
 0.0010648724763652549,
 0.0010496857996565646,
 0.0013014313103919186,
 0.0012388406530434166,
 0.0012607404623400603,
 0.0013181275479065473,
 0.0011095479771857984,
 0.001297290637738735,
 0.001260259769782249,
 0.0012702461787780016,
 0.0011304991892252595,
 0.0012348504610391697,
 0.001174679439886562,
 0.0012746568189116734,
 0.001269108683132162,
 0.0012595264559163862,
 0.001211360094348005,
 0.0012800281071044443,
 0.0012171870660383152,
 0.0009290415578126872,
 0.0012355445070553472,
 0.0013499227245067284,
 0.0011418088821503825,
 0.001262775878329862,
 0.0013065211362662612,
 0.0012174801427897743,
 0.001251371856360455,
 0.001316161773397245,
 0.0012611464405053922,
 0.0012743489643784725,
 0.0011279752926316652,
 0.0012005245895042045,
 0.0011032736416612648,
 0.0009899230502265267,
 0

In [31]:
# 3a) Computing the Scatter Matrix
scatter_matrix = np.zeros((corpus_data.shape[0],corpus_data.shape[0]))
for i in range(corpus_data.shape[1]):
    scatter_matrix += (corpus_data[:,i].reshape(corpus_data.shape[0],1) - mean_vector).dot((corpus_data[:,i].reshape(corpus_data.shape[0],1) - mean_vector).T)
scatter_matrix

array([[290.80674447, 233.44789293, 229.03395872, ..., 233.42584034,
        231.41154806, 188.65474194],
       [233.44789293, 281.68064924, 242.93154907, ..., 252.6532779 ,
        264.70040672, 211.79967353],
       [229.03395872, 242.93154907, 281.07477635, ..., 237.5522456 ,
        246.7720511 , 190.81101666],
       ...,
       [233.42584034, 252.6532779 , 237.5522456 , ..., 286.33372698,
        247.75373865, 207.14963227],
       [231.41154806, 264.70040672, 246.7720511 , ..., 247.75373865,
        282.97809122, 207.38616715],
       [188.65474194, 211.79967353, 190.81101666, ..., 207.14963227,
        207.38616715, 294.0826969 ]])

In [33]:
# 3b) Computing the Covariance Matrix
cov_mat = np.cov(corpus_data)
cov_mat

array([[1.24313339e-05, 9.99071046e-06, 9.80275229e-06, ...,
        9.98277137e-06, 9.90162615e-06, 8.05539863e-06],
       [9.99071046e-06, 1.20445615e-05, 1.03858521e-05, ...,
        1.08075967e-05, 1.13194100e-05, 9.06798621e-06],
       [9.80275229e-06, 1.03858521e-05, 1.20172052e-05, ...,
        1.01615737e-05, 1.05518290e-05, 8.17137280e-06],
       ...,
       [9.98277137e-06, 1.08075967e-05, 1.01615737e-05, ...,
        1.22476894e-05, 1.05976508e-05, 8.85767862e-06],
       [9.90162615e-06, 1.13194100e-05, 1.05518290e-05, ...,
        1.05976508e-05, 1.21024404e-05, 8.87597468e-06],
       [8.05539863e-06, 9.06798621e-06, 8.17137280e-06, ...,
        8.85767862e-06, 8.87597468e-06, 1.25586037e-05]])

In [34]:
# 4) Computing eigenvectors and corresponding eigenvalues
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(scatter_matrix)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)

for i in range(len(eig_val_sc)):
    eigvec_sc = eig_vec_sc[:,i].reshape(1,corpus_data.shape[0]).T
    eigvec_cov = eig_vec_cov[:,i].reshape(1,corpus_data.shape[0]).T
    assert eigvec_sc.all() == eigvec_cov.all(), 'Eigenvectors are not identical'

    print('Eigenvector {}: \n{}'.format(i+1, eigvec_sc))
    print('Eigenvalue {} from scatter matrix: {}'.format(i+1, eig_val_sc[i]))
    print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))
    print('Scaling factor: ', eig_val_sc[i]/eig_val_cov[i])
    print(40 * '-')


Eigenvector 1: 
[[-0.05262136]
 [-0.05835499]
 [-0.0553277 ]
 [-0.04999109]
 [-0.05458479]
 [-0.04694219]
 [-0.05655274]
 [-0.04606327]
 [-0.04108362]
 [-0.04385237]
 [-0.05228643]
 [-0.0464544 ]
 [-0.05434811]
 [-0.05694958]
 [-0.04132667]
 [-0.05663358]
 [-0.05532235]
 [-0.05381899]
 [-0.04529663]
 [-0.05237525]
 [-0.04367437]
 [-0.0541269 ]
 [-0.05515831]
 [-0.04926551]
 [-0.05050901]
 [-0.04880101]
 [-0.05271431]
 [-0.03731287]
 [-0.05374669]
 [-0.05777527]
 [-0.04959313]
 [-0.05621484]
 [-0.05677561]
 [-0.05196791]
 [-0.05453848]
 [-0.05289323]
 [-0.05098997]
 [-0.0550483 ]
 [-0.04723135]
 [-0.05284137]
 [-0.04644657]
 [-0.04391174]
 [-0.0547085 ]
 [-0.05713523]
 [-0.03971813]
 [-0.05488658]
 [-0.05545468]
 [-0.05467861]
 [-0.0470063 ]
 [-0.05797776]
 [-0.05756402]
 [-0.05259275]
 [-0.0480944 ]
 [-0.05711472]
 [-0.0561636 ]
 [-0.05578853]
 [-0.05042256]
 [-0.05419195]
 [-0.04830608]
 [-0.04645876]
 [-0.05369963]
 [-0.05236341]
 [-0.05556859]
 [-0.04791326]
 [-0.04413032]
 [-0.0461

 [-0.06289733]]
Eigenvalue 55 from scatter matrix: 68.73126973528933
Eigenvalue 55 from covariance matrix: 2.8731468579608997e-06
Scaling factor:  23921948.000969425
----------------------------------------
Eigenvector 56: 
[[ 6.42946800e-02]
 [ 3.15973302e-02]
 [-1.36219284e-01]
 [ 1.56977933e-01]
 [ 1.61649070e-01]
 [-9.59500080e-02]
 [-3.85604493e-03]
 [ 7.41333283e-02]
 [ 4.06832380e-02]
 [-4.61072783e-02]
 [-1.00552751e-01]
 [-7.23865546e-02]
 [ 2.70591283e-02]
 [ 5.77543195e-02]
 [-6.87453663e-02]
 [ 5.14784171e-02]
 [ 5.60428423e-03]
 [ 1.20095359e-02]
 [-2.14940309e-02]
 [-2.38737220e-01]
 [ 6.49429503e-02]
 [-7.35063952e-02]
 [-4.22106460e-02]
 [-7.58820726e-02]
 [-6.91911862e-02]
 [ 1.57122074e-01]
 [-2.51804487e-02]
 [-1.79409039e-03]
 [ 5.40540769e-02]
 [ 2.64427417e-02]
 [-3.60846639e-02]
 [-5.72428145e-02]
 [-4.18120810e-02]
 [ 2.13185326e-02]
 [ 2.33549176e-02]
 [ 3.04644149e-02]
 [-8.35970429e-02]
 [ 5.43972759e-02]
 [-1.10827537e-01]
 [-4.33038751e-03]
 [ 1.13642618e-0

 [ 3.31672865e-02]]
Eigenvalue 121 from scatter matrix: 28.506696047534856
Eigenvalue 121 from covariance matrix: 1.2255407210865434e-06
Scaling factor:  23260504.981230903
----------------------------------------
Eigenvector 122: 
[[ 4.14626876e-02]
 [ 3.10558686e-02]
 [ 4.98116864e-02]
 [-3.88835157e-02]
 [ 8.71558671e-03]
 [ 1.36297125e-02]
 [ 7.26088564e-02]
 [ 2.16061593e-02]
 [ 5.30292158e-02]
 [ 1.90574530e-02]
 [-4.95530924e-02]
 [ 5.81376453e-03]
 [-8.05869833e-03]
 [-8.88262315e-02]
 [-5.88213581e-03]
 [ 3.17401150e-02]
 [ 1.79372808e-02]
 [ 1.22032328e-02]
 [ 2.72286829e-02]
 [-8.11293722e-02]
 [ 5.23576482e-03]
 [-2.68710399e-02]
 [ 1.25759335e-02]
 [-3.99953608e-03]
 [ 5.85354978e-02]
 [-1.61233894e-02]
 [-4.32791606e-02]
 [ 3.09722369e-03]
 [-6.32984532e-02]
 [ 5.64471967e-03]
 [-3.76125406e-02]
 [-8.08153140e-02]
 [ 5.31769743e-03]
 [ 2.90274086e-02]
 [-1.21390931e-01]
 [-9.57577219e-02]
 [-1.44357126e-02]
 [-2.33271662e-01]
 [ 4.24255536e-02]
 [-2.50467091e-02]
 [-4.812

 [-2.63683126e-03]]
Eigenvalue 174 from scatter matrix: 1.3694348953486015
Eigenvalue 174 from covariance matrix: 6.679798001836439e-07
Scaling factor:  2050114.2324545002
----------------------------------------
Eigenvector 175: 
[[ 4.05120920e-03]
 [-1.13949528e-02]
 [ 3.95356809e-03]
 [-1.03022810e-04]
 [ 3.37699480e-03]
 [-3.77756037e-04]
 [-2.20521246e-03]
 [ 4.95154309e-03]
 [-4.48373807e-03]
 [ 3.39759583e-03]
 [ 5.88737388e-03]
 [ 5.27257614e-03]
 [-7.39752716e-04]
 [-1.14609699e-02]
 [ 9.09508380e-05]
 [ 2.31742188e-02]
 [-1.00386964e-02]
 [ 9.26282216e-03]
 [ 6.29359315e-03]
 [ 4.64530817e-03]
 [ 2.50753866e-03]
 [ 3.64186735e-03]
 [-2.95292919e-02]
 [ 1.48751762e-02]
 [-3.66229813e-03]
 [-3.95548024e-03]
 [ 1.51594811e-02]
 [ 5.98540572e-03]
 [-7.46760972e-03]
 [-5.04126516e-03]
 [ 8.25085425e-04]
 [-6.89401635e-03]
 [-1.00438785e-02]
 [ 1.15538070e-02]
 [ 3.71785436e-03]
 [ 7.87388703e-04]
 [-3.85550985e-04]
 [-4.77631816e-03]
 [ 1.16149340e-03]
 [ 3.60196182e-03]
 [ 6.3547

 [-1.04363898e-02]]
Eigenvalue 243 from scatter matrix: 10.062010352565663
Eigenvalue 243 from covariance matrix: 4.382301832266809e-07
Scaling factor:  22960559.855734408
----------------------------------------
Eigenvector 244: 
[[-4.93013140e-03]
 [ 5.19553499e-02]
 [-5.04298581e-02]
 [ 3.38948834e-03]
 [ 4.59601831e-02]
 [ 4.30880230e-03]
 [ 5.43983999e-02]
 [ 3.40310968e-03]
 [ 6.18421287e-03]
 [-1.00921092e-02]
 [ 2.94754509e-02]
 [ 1.45406638e-02]
 [-1.93692052e-02]
 [ 4.06978076e-02]
 [-2.70583717e-03]
 [-6.14266133e-02]
 [ 1.77491026e-02]
 [-1.75108139e-02]
 [ 7.30006400e-03]
 [ 2.51936530e-02]
 [-1.85948943e-02]
 [ 2.96218787e-02]
 [ 1.01794373e-03]
 [-2.28054015e-02]
 [-1.31468801e-03]
 [ 9.31980827e-03]
 [ 3.28357372e-02]
 [ 2.63566445e-03]
 [-7.27960454e-03]
 [ 1.02964720e-01]
 [ 8.32098118e-03]
 [-4.30974709e-02]
 [-4.52736713e-02]
 [ 1.41610638e-02]
 [ 1.92448627e-02]
 [-5.20763326e-02]
 [ 3.22996165e-02]
 [-2.50010599e-02]
 [-1.20792397e-02]
 [ 4.76101157e-03]
 [-4.2642

 [ 5.13254531e-03]]
Eigenvalue 298 from scatter matrix: 5.3901461404491
Eigenvalue 298 from covariance matrix: 1.8651510223158722e-07
Scaling factor:  28899247.706796438
----------------------------------------
Eigenvector 299: 
[[-4.53183917e-02]
 [-1.19101197e-02]
 [-1.44306624e-02]
 [ 4.57090754e-03]
 [-1.73060400e-02]
 [ 8.88860231e-03]
 [ 4.90239552e-03]
 [ 5.64375789e-03]
 [ 3.05873855e-03]
 [ 2.25533038e-03]
 [ 3.42461269e-02]
 [ 2.71044596e-03]
 [-2.01807963e-03]
 [ 6.66901942e-03]
 [ 1.36316854e-02]
 [ 2.11459579e-02]
 [-4.17095392e-02]
 [-1.12258831e-02]
 [ 6.68797597e-03]
 [-1.02383745e-02]
 [-1.85473541e-02]
 [ 1.57587310e-02]
 [-1.73403780e-01]
 [ 1.66259137e-02]
 [-8.28170144e-03]
 [ 3.86322841e-04]
 [ 7.41242841e-03]
 [-1.13352115e-02]
 [-1.35612372e-02]
 [ 1.42162743e-02]
 [ 5.08725568e-03]
 [ 1.22129014e-02]
 [-1.53021221e-02]
 [-4.61672731e-03]
 [ 5.01868879e-03]
 [-1.03300497e-02]
 [-2.15559806e-02]
 [ 2.96538370e-02]
 [-3.39492470e-03]
 [-9.88800216e-03]
 [ 9.109544

In [37]:
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,corpus_data.shape[0]).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv), eig_val_sc[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)


In [38]:
# 5.1) Sorting the eigenvectors by decreasing eigenvalues
for ev in eig_vec_sc:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

AssertionError: 
Arrays are not almost equal to 6 decimals

(mismatch 100.0%)
 x: array(1.)
 y: array(1.011857)

In [39]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_val_sc[i]), eig_vec_sc[:,i]) for i in range(len(eig_val_sc))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
for i in eig_pairs:
    print(i[0])


76709.37777676433
1834.3258429276439
917.801178985962
710.1441180200225
691.4454948900868
367.2452304478882
285.8131173976129
244.9048255382587
207.46701898469436
190.64546211383498
183.8894804360585
176.96978277388874
171.41497451090714
163.87138610693646
158.06297215009343
154.97371808318897
152.11249954455016
147.0371562305428
143.1527740707503
135.72622671151714
134.04778809407563
129.80732765306539
126.24840170751068
124.20501264400428
122.54372195487787
119.89408947367956
117.11670936359995
115.5130287393547
111.59566613713845
111.14632200534955
106.9866608531808
104.80372245877545
102.35862685273281
100.97322825962718
98.91494971414497
95.57062891234138
92.62208611151851
92.04522944912772
90.85407139803193
88.52429089200464
87.4024275412983
84.37540294218338
83.69804784614105
82.74040631297595
81.32646582274155
79.81906544085247
78.4518089748733
77.16574401203373
75.79497431951405
74.59092917759871
73.85059931779082
72.41863259983289
71.58438270272602
69.7016654289979
68.7312697

In [40]:
# 5.2) Choosing k eigenvectors with the largest eigenvalues
matrix_w = np.hstack((eig_pairs[0][1].reshape(corpus_data.shape[0],1), eig_pairs[1][1].reshape(corpus_data.shape[0],1)))
print('Matrix W:\n', matrix_w)


Matrix W:
 [[-5.26213621e-02  2.40378000e-02]
 [-5.83549860e-02 -1.21915202e-02]
 [-5.53276980e-02  3.54651197e-03]
 [-4.99910903e-02 -9.03659615e-02]
 [-5.45847882e-02 -2.58560551e-02]
 [-4.69421946e-02 -4.72045482e-02]
 [-5.65527406e-02  1.49970413e-02]
 [-4.60632667e-02  2.60154851e-04]
 [-4.10836151e-02 -5.45912405e-02]
 [-4.38523701e-02 -6.16949887e-03]
 [-5.22864302e-02 -9.90367699e-02]
 [-4.64543962e-02 -3.18439385e-02]
 [-5.43481059e-02 -1.52963352e-02]
 [-5.69495791e-02  3.68615201e-02]
 [-4.13266653e-02  6.65168732e-03]
 [-5.66335752e-02  3.96724606e-03]
 [-5.53223543e-02  8.22681789e-02]
 [-5.38189911e-02  1.14496390e-02]
 [-4.52966260e-02 -2.71866013e-02]
 [-5.23752541e-02 -3.49590134e-03]
 [-4.36743682e-02 -2.39555982e-02]
 [-5.41268973e-02 -1.51044656e-03]
 [-5.51583064e-02  8.95418905e-02]
 [-4.92655077e-02 -7.79691879e-02]
 [-5.05090142e-02 -1.17129342e-02]
 [-4.88010129e-02 -5.03410759e-03]
 [-5.27143104e-02 -1.13749847e-01]
 [-3.73128691e-02  2.54536063e-02]
 [-5.3746

In [48]:
# 6) Transforming the samples onto the new subspace
transformed = matrix_w.T.dot(corpus_data)
# assert transformed.shape == (2,), "The matrix is not 2x40 dimensional."
transformed.shape

(2, 72355)

In [43]:
from matplotlib.mlab import PCA as mlabPCA
mlab_pca = mlabPCA(corpus_data.T)

In [54]:
from sklearn.manifold import TSNE
corpus_data_embedded = TSNE(n_components=3).fit_transform(corpus_data)

In [56]:
corpus_data_embedded

array([[ 1.51177353e+02,  2.33076439e+01, -1.07699501e+02],
       [-1.05526295e+01,  2.11963862e-01,  2.39670372e+01],
       [ 9.98975983e+01, -7.06171799e+01, -1.66687870e+01],
       [-2.08724632e+01, -5.96292610e+01,  1.61089691e+02],
       [ 4.37796326e+01,  1.58510574e+02,  5.05506630e+01],
       [-5.01641960e+01, -4.52102394e+01,  1.20845108e+02],
       [ 1.67467239e+02,  5.73462906e+01, -5.71340256e+01],
       [-1.30331783e+01, -1.24120064e+02,  2.02932358e+01],
       [-6.82667923e+01,  1.21241524e+02,  2.06537838e+01],
       [ 1.05458359e+02, -1.42712814e+02,  5.00961761e+01],
       [ 2.17640629e+01, -4.31818733e+01,  7.29839325e+01],
       [ 3.69687920e+01, -6.98688965e+01,  8.96365509e+01],
       [-1.05632599e+02,  1.53089493e+02, -1.67149429e+01],
       [ 9.35414124e+01, -1.09515045e+02, -5.92132187e+01],
       [ 7.22959459e-01, -5.39334488e+01,  3.27548218e+01],
       [ 1.75011330e+01, -1.61055695e+02,  3.18618565e+01],
       [ 7.02698898e+01, -9.39209290e+01